In [1]:
# Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter # to print to tensorboard
from tqdm import tqdm

In [2]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [3]:
# Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
#Hyperparameters
# learning_rate = 0.001
in_channels = 1
num_classes = 10
# batch_size = 64
num_epochs = 1

In [5]:
#load data
train_dataset = datasets.MNIST(
    root='dataset/',
    train=True,
    transform=transforms.ToTensor(),
    download=False
)

In [15]:
batch_sizes = [256]
learning_rates = [0.01]
classes = [str(x) for x in range(10)] #class labels

In [25]:

for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        step = 0

        ## move elements inside the loop which depends on changing hyperparameters

        #new trainloader according to batch size
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)
        
        #initialize model
        model = CNN(in_channels=in_channels,num_classes=num_classes).to(device=device)
        model.train()
        
        # initialize summaryWriter for each combo
        writer = SummaryWriter(f"runs/MNIST/MiniBatchSize-{batch_size}-LR-{learning_rate}")

        # new loss_fn and optimizer with changed hyperparamters
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(),lr=learning_rate, weight_decay=0.0)

        for epoch in range(num_epochs):
            losses = []
            accuracies = []
            for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
                data = data.to(device)
                targets = targets.to(device)

                #forward
                score = model(data)
                loss = loss_fn(score,targets)
                losses.append(loss.item())

                #backward
                optimizer.zero_grad()
                loss.backward()

                #optimizer step
                optimizer.step()


                #Calculate 'running accuracy'
                _,predictions = score.max(1)
                num_correct = (predictions==targets).sum()
                running_train_acc = float(num_correct)/float(data.shape[0])
                accuracies.append(running_train_acc)
                
                ### Plotting to tensorboard 
                # Plotting batch image grid
                img_grid = torchvision.utils.make_grid(data)
                writer.add_image('mnist_images', img_grid,global_step=step)

                #writing weights of any layer like fc1 or conv1
                writer.add_histogram('fc1',model.fc1.weight,global_step=step)
                writer.add_histogram('conv1',model.conv1.weight,global_step=step)

                 
                # plotting loss & acc to tensorboard
                writer.add_scalar('Training loss',loss,global_step=step)
                writer.add_scalar('Training accuracy:',running_train_acc,global_step=step)

                # plotting embdeddings 

                if batch_idx == 233:
                    features = data.reshape(data.shape[0],-1)
                    class_labels = [classes[label] for label in predictions]
                    writer.add_embedding(
                        features,
                        metadata=class_labels,
                        label_img = data,
                        global_step=batch_idx)

                
                #update global step
                step+=1
            
            # write hparams after each batch
            writer.add_hparams({'lr': learning_rate,'bsize':batch_size},
                               {'accuracy': sum(accuracies)/len(accuracies),
                                'loss':sum(losses)/len(losses)})

235it [00:44,  5.24it/s]


In [ ]:
# check tensorboard by 'python -m tensorboard.main --logdir runs'